# Phase 5: Meta-Supervisor Demonstration

This notebook demonstrates the **Meta-Supervisor** - the orchestrator that:
1. Collects predictions from all 6 specialized agents
2. Synthesizes them into a coherent investment hypothesis
3. Provides a weighted prediction with confidence score

**6 Specialized Agents:**
- **Core Agents (3)**: Fundamentals, Valuation, Technical
- **Context Agents (3)**: Macro, Events, Sector

## Setup and Imports

In [ ]:
import sys
import os
from datetime import date, timedelta

# Clear any cached src modules
modules_to_clear = [k for k in list(sys.modules.keys()) if k.startswith('src.')]
for mod in modules_to_clear:
    del sys.modules[mod]

# Auto-detect repository path
repo_path = None
try:
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    if '/Repos/' in notebook_path:
        parts = notebook_path.split('/Repos/')
        if len(parts) > 1:
            repo_base = '/Workspace/Repos/' + parts[1].split('/')[0] + '/stocks-ai-system'
            if os.path.exists(repo_base):
                repo_path = repo_base
except:
    pass

if not repo_path:
    possible_paths = [
        '/Workspace/Repos/stocks-ai-system',
        '/Workspace/Users/' + os.getenv('USER', 'user') + '/stocks-ai-system',
    ]
    for path in possible_paths:
        if os.path.exists(path):
            repo_path = path
            break

if repo_path and repo_path not in sys.path:
    sys.path.insert(0, repo_path)
    print(f"✓ Added {repo_path} to Python path")
elif repo_path:
    print(f"✓ Path already configured: {repo_path}")
else:
    print("⚠ Could not auto-detect repository path. Please set repo_path manually.")

In [ ]:
# Import Meta-Supervisor
from src.agents.meta_supervisor import MetaSupervisor

print("✓ Meta-Supervisor imported successfully")

## Test Configuration

In [ ]:
# Test configuration
TEST_SYMBOL = "AAPL"  # Apple Inc.
TEST_DATE = date.today()
TARGET_DATE = TEST_DATE + timedelta(days=30)  # 30-day prediction

print(f"Stock Symbol: {TEST_SYMBOL}")
print(f"Prediction Date: {TEST_DATE}")
print(f"Target Date: {TARGET_DATE}")
print(f"Prediction Horizon: {(TARGET_DATE - TEST_DATE).days} days")

## Initialize Meta-Supervisor

The Meta-Supervisor initializes all 6 specialized agents

In [ ]:
print("Initializing Meta-Supervisor...\n")
print("=" * 80)

supervisor = MetaSupervisor()

print("=" * 80)
print(f"\n✅ Meta-Supervisor ready with {len(supervisor.agents)} specialized agents")

## View Agent Summary

In [ ]:
print("📋 AGENT SUMMARY")
print("=" * 80)

agent_summary = supervisor.get_agent_summary()

for i, (agent_name, info) in enumerate(agent_summary.items(), 1):
    print(f"\n{i}. {info['name'].upper()}")
    print(f"   {info['description']}")

print("\n" + "=" * 80)

## Generate Synthesized Prediction

This is the main orchestration:
1. Meta-Supervisor calls all 6 agents
2. Each agent collects signals and generates predictions
3. Meta-Supervisor synthesizes all predictions
4. Final investment hypothesis is generated

In [ ]:
print("=" * 80)
print("GENERATING SYNTHESIZED PREDICTION")
print("=" * 80)
print(f"\nStock: {TEST_SYMBOL}")
print(f"Horizon: {(TARGET_DATE - TEST_DATE).days} days\n")

print("⏳ Orchestrating all 6 agents...\n")

# Generate synthesized prediction
synthesized = supervisor.generate_prediction(
    symbol=TEST_SYMBOL,
    as_of_date=TEST_DATE,
    target_date=TARGET_DATE
)

print("\n" + "=" * 80)
print("✅ SYNTHESIS COMPLETE")
print("=" * 80)

## View Individual Agent Contributions

In [ ]:
print("\n📊 INDIVIDUAL AGENT PREDICTIONS")
print("=" * 80)

for agent_name, contribution in synthesized.agent_contributions.items():
    print(f"\n🤖 {agent_name.upper()} Agent:")
    print(f"   Predicted Return: {contribution['predicted_return']:+.2f}%")
    print(f"   Confidence: {contribution['confidence']:.2%}")
    print(f"   Reasoning (first 150 chars):")
    print(f"   {contribution['reasoning'][:150]}...")

print("\n" + "=" * 80)

## View Synthesized Results

In [ ]:
print("\n🎯 SYNTHESIZED PREDICTION")
print("=" * 80)
print(f"\nStock: {synthesized.symbol}")
print(f"Prediction Date: {synthesized.prediction_date}")
print(f"Target Date: {synthesized.target_date}")
print(f"\n📈 Predicted Return: {synthesized.predicted_return:+.2f}%")
print(f"🎲 Confidence Score: {synthesized.confidence_score:.2%}")

print("\n" + "=" * 80)

In [ ]:
print("\n💡 INVESTMENT HYPOTHESIS")
print("=" * 80)
print(f"\n{synthesized.investment_hypothesis}")
print("\n" + "=" * 80)

In [ ]:
print("\n🔍 KEY INSIGHTS")
print("=" * 80)

if synthesized.key_insights:
    for i, insight in enumerate(synthesized.key_insights, 1):
        print(f"{i}. {insight}")
else:
    print("(See reasoning chain below)")

print("\n" + "=" * 80)

In [ ]:
print("\n⚠️  RISK FACTORS")
print("=" * 80)

if synthesized.risk_factors:
    for i, risk in enumerate(synthesized.risk_factors, 1):
        print(f"{i}. {risk}")
else:
    print("(See reasoning chain below)")

print("\n" + "=" * 80)

In [ ]:
print("\n🧠 REASONING CHAIN")
print("=" * 80)
print(f"\n{synthesized.reasoning_chain}")
print("\n" + "=" * 80)

## Summary Statistics

In [ ]:
print("\n📊 PREDICTION SUMMARY")
print("=" * 80)

# Calculate statistics
predictions = [c['predicted_return'] for c in synthesized.agent_contributions.values()]
confidences = [c['confidence'] for c in synthesized.agent_contributions.values()]

print(f"\nNumber of Agents: {len(synthesized.agent_contributions)}")
print(f"\nIndividual Predictions:")
print(f"  Min: {min(predictions):+.2f}%")
print(f"  Max: {max(predictions):+.2f}%")
print(f"  Mean: {sum(predictions)/len(predictions):+.2f}%")
print(f"  Range: {max(predictions) - min(predictions):.2f}%")

print(f"\nConfidence Scores:")
print(f"  Min: {min(confidences):.2%}")
print(f"  Max: {max(confidences):.2%}")
print(f"  Mean: {sum(confidences)/len(confidences):.2%}")

print(f"\n🎯 Synthesized Prediction: {synthesized.predicted_return:+.2f}%")
print(f"🎲 Synthesized Confidence: {synthesized.confidence_score:.2%}")

# Agreement metric
bullish_agents = sum(1 for p in predictions if p > 0)
bearish_agents = sum(1 for p in predictions if p < 0)
neutral_agents = sum(1 for p in predictions if p == 0)

print(f"\nAgent Consensus:")
print(f"  Bullish: {bullish_agents}/{len(predictions)} agents")
print(f"  Bearish: {bearish_agents}/{len(predictions)} agents")
print(f"  Neutral: {neutral_agents}/{len(predictions)} agents")

if bullish_agents == len(predictions):
    print(f"\n  📈 Unanimous Bullish Consensus")
elif bearish_agents == len(predictions):
    print(f"\n  📉 Unanimous Bearish Consensus")
elif bullish_agents > bearish_agents:
    print(f"\n  📊 Bullish Majority ({bullish_agents}/{len(predictions)})")
elif bearish_agents > bullish_agents:
    print(f"\n  📊 Bearish Majority ({bearish_agents}/{len(predictions)})")
else:
    print(f"\n  ⚖️  Split Decision")

print("\n" + "=" * 80)

## Phase 5 Complete! 🎉

### What We Demonstrated:

1. **Meta-Supervisor Orchestration**: Successfully coordinated all 6 specialized agents
2. **Multi-Perspective Analysis**: Each agent analyzed the stock from their expertise
3. **Intelligent Synthesis**: Combined predictions into coherent investment hypothesis
4. **Confidence Weighting**: More confident agents have greater influence
5. **Risk Assessment**: Identified key risks and opportunities

### Next Steps:

- **Phase 6**: Risk Agent & Portfolio optimization
- **Phase 7**: ESG Agent & Sustainability analysis
- **Phase 8**: Real-time streaming predictions
- **Phase 9**: Backtesting framework
- **Phase 10**: MLflow integration & model governance

In [ ]:
print("\n" + "=" * 80)
print("✅ PHASE 5 COMPLETE!")
print("=" * 80)
print("\nMeta-Supervisor successfully synthesized predictions from 6 agents:")
print("  • Fundamentals Agent")
print("  • Valuation Agent")
print("  • Technical Agent")
print("  • Macro Agent")
print("  • Events Agent")
print("  • Sector Agent")
print("\nNext: Phase 6 - Risk Agent & Portfolio Management")
print("=" * 80)